Import Lib

In [1]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [2]:
class stepState(TypedDict):
  input:str
  step1:str
  step2:str
  step3:str
  step4:str

In [3]:
def step1(state:stepState)->stepState:
  print("Step-1 Executed")
  return {'step1':'Done', 'input':state['input']}
  

def step2(state:stepState)->stepState:
  print("Step-2 Executed")
  return {'step2':'Done'}

def step3(state:stepState)->stepState:
  print("Step-3 hanging..now interrupt (STOP BUTTON)")
  time.sleep(5) # long running hang
  return {'step3':'Done'}

def step4(state:stepState)->stepState:
  print("Step-4 Executed")
  return {'step4':True}


In [4]:
graph = StateGraph(stepState)

graph.add_node('Step 1', step1)
graph.add_node('Step 2', step2)
graph.add_node('Step 3', step3)
graph.add_node('Step 4', step4)

graph.add_edge(START, 'Step 1')
graph.add_edge('Step 1', 'Step 2')
graph.add_edge('Step 2', 'Step 3')
graph.add_edge('Step 3', 'Step 4')
graph.add_edge('Step 4', END)

checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer=checkpointer)

In [5]:
try:
  print("Running graph: Please manually interrupt during the step 3")
  workflow.invoke({'input':'START'}, config={'configurable':{'thread_id':'thread-1'}})
except KeyboardInterrupt:
  print("Kernal manually interrupt (crash simulated)")

Running graph: Please manually interrupt during the step 3
Step-1 Executed
Step-2 Executed
Step-3 hanging..now interrupt (STOP BUTTON)
Kernal manually interrupt (crash simulated)


In [6]:
workflow.get_state({'configurable':{'thread_id':'thread-1'}})

StateSnapshot(values={'input': 'START', 'step1': 'Done', 'step2': 'Done'}, next=('Step 3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08a153-25af-62dc-8002-f33b746a0f31'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-05T05:00:09.225242+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08a153-25a6-6e0c-8001-3421f68cd604'}}, tasks=(PregelTask(id='c739ea30-63e1-4565-8d57-c5bcadcac7a7', name='Step 3', path=('__pregel_pull', 'Step 3'), error=None, interrupts=(), state=None, result=None),), interrupts=())

In [7]:
list(workflow.get_state_history({'configurable':{'thread_id':'thread-1'}}))

[StateSnapshot(values={'input': 'START', 'step1': 'Done', 'step2': 'Done'}, next=('Step 3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08a153-25af-62dc-8002-f33b746a0f31'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-05T05:00:09.225242+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08a153-25a6-6e0c-8001-3421f68cd604'}}, tasks=(PregelTask(id='c739ea30-63e1-4565-8d57-c5bcadcac7a7', name='Step 3', path=('__pregel_pull', 'Step 3'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'input': 'START', 'step1': 'Done'}, next=('Step 2',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08a153-25a6-6e0c-8001-3421f68cd604'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-09-05T05:00:09.221816+00:00', parent_config={'configurable': {'thread_id': 't

### Re-run the state to show the Fault-Tolerant

In [8]:
final_state = workflow.invoke(None, config={'configurable':{'thread_id':'thread-1'}})

Step-3 hanging..now interrupt (STOP BUTTON)
Step-4 Executed


In [10]:
workflow.get_state({'configurable':{'thread_id':'thread-1'}})

StateSnapshot(values={'input': 'START', 'step1': 'Done', 'step2': 'Done', 'step3': 'Done', 'step4': True}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08a153-dbcf-6810-8004-256ef228d93b'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}}, created_at='2025-09-05T05:00:28.322576+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08a153-dbca-6dde-8003-0d0413d47751'}}, tasks=(), interrupts=())

In [11]:
list(workflow.get_state_history({'configurable':{'thread_id':'thread-1'}}))

[StateSnapshot(values={'input': 'START', 'step1': 'Done', 'step2': 'Done', 'step3': 'Done', 'step4': True}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08a153-dbcf-6810-8004-256ef228d93b'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}}, created_at='2025-09-05T05:00:28.322576+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08a153-dbca-6dde-8003-0d0413d47751'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'START', 'step1': 'Done', 'step2': 'Done', 'step3': 'Done'}, next=('Step 4',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08a153-dbca-6dde-8003-0d0413d47751'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-09-05T05:00:28.320668+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08a153-25af-62dc-8002-f33b746a0f31'}}, tasks=(P